In [ ]:
from sklearn.model_selection import GridSearchCV
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation
from tensorflow.keras.callbacks import EarlyStopping
from keras.utils.vis_utils import plot_model
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier 
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn import tree
from sklearn.metrics import confusion_matrix
from matplotlib import pyplot as pyplot
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import plot_confusion_matrix
from warnings import simplefilter
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
import time
start = time.time()


dftrain = pd.read_csv("../input/mqttset/Data/FINAL_CSV/train70_reduced.csv") 
dftest = pd.read_csv("../input/datarev4/test30_reduced.csv")
simplefilter(action='ignore', category=FutureWarning)
seed = 7

#train
#print(dftrain.loc[dftrain['target'] == 'legitimate'])
print(dftrain['target'])
print(dftrain['target'].value_counts())
class_names = dftrain.target.unique()
dftrain=dftrain.astype('category')
cat_columns = dftrain.select_dtypes(['category']).columns
dftrain[cat_columns] = dftrain[cat_columns].apply(lambda x: x.cat.codes)
#print(dftrain.loc[125, 'target'])
x_columns = dftrain.columns.drop('target')
x_train = dftrain[x_columns].values
y_train = dftrain['target']

#test

class_names = dftest.target.unique()
dftest=dftest.astype('category')
cat_columns = dftest.select_dtypes(['category']).columns
dftest[cat_columns] = dftest[cat_columns].apply(lambda x: x.cat.codes)
x_columns = dftest.columns.drop('target')
x_test = dftest[x_columns].values
y_test = dftest['target']


print("Ready to generate train and test datasets")

In [ ]:
#Visualization
def bar_graph(feature):
    dftest[feature].value_counts().plot(kind="bar")

In [ ]:
plt.figure(figsize=[15, 10])
# Data to be plotted
totalDeath = [140000, 140000, 140000, 700000, 140000,5000]
totalRecovery = [1000, 1000, 1000, 1000, 1000,1000]
activeCases = [1139958, 347973, 239999, 129360, 34730,34730]
# Using numpy to group 3 different data with bars
X = np.arange(len(totalDeath))
# Passing the parameters to the bar function, this is the main function which creates the bar plot
# Using X now to align the bars side by side
plt.bar(X, totalDeath, color = 'blue', width = 0.25)
# Creating the legend of the bars in the plot
# Overiding the x axis with the country names
plt.xticks([i + 0.0 for i in range(6)], ['Dos', 'Bruteforce', 'SlowITE','Legitimate','Malformed','MITM'],fontsize=18, fontweight='bold')
# Giving the tilte for the plot
plt.title("Saldırı ve Veri Sayılarının Karşılaştırılması",fontsize=18, fontweight='bold')
# Namimg the x and y axis
plt.ylabel('Adet')
# Saving the plot as a 'png'
plt.savefig('4BarPlot.png')
# Displaying the bar plot
plt.show()

In [ ]:
countries = ['Dos', 'Bruteforce', 'SlowITE','Legitimate','Malformed']
# Integer value interms of death counts
totalDeaths = [140000,140000,140000,700000,140000]
# Passing the parameters to the bar function, this is the main function which creates the bar plot
plt.bar(countries, totalDeaths)
# Displaying the bar plot
plt.show()

plt.title("Saldırı ve Veri Sayılarının Karşılaştırılması")
# Namimg the x and y axis
plt.ylabel('Sayı')


In [ ]:
x = np.arange(5)
plt.bar(x, height=[1400000,1400000,1400000,7000000,1400000])
plt.xticks(x, ['Dos', 'Bruteforce', 'SlowITE','Legitimate','Malformed'])

In [ ]:

#Neural network
print("Starting Random forest")
model = Sequential()
model.add(Dense(50, input_dim=x_train.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(30, input_dim=x_train.shape[1], kernel_initializer='normal', activation='relu'))
model.add(Dense(20, kernel_initializer='normal'))
model.add(Dense(6,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=5, verbose=1, mode='auto')
history = model.fit(x_train,y_train,validation_data=(x_test,y_test),callbacks=[monitor],verbose=2,epochs=200,batch_size=1000) 
end = time.time()
diff=end-start
print("Training time: " + str(diff))
starttest = time.time()
y_pred_nn = model.predict(x_test)
y_pred_nn = np.argmax(y_pred_nn,axis=1)
endtest =time.time()
difftest = endtest-starttest
print("Test time: " + str(difftest))

print(model.summary())


#RandomForest
print("Starting Random forest")
classifier = RandomForestClassifier(verbose=2,random_state=seed)
classifier.fit(x_train, y_train)
end = time.time()
diff=end-start
print("Training time: " + str(diff))
starttest = time.time()
y_pred_random = classifier.predict(x_test)
endtest =time.time()
difftest = endtest-starttest
print("Test time: " + str(difftest))

#Create Naive Bayes Classifier
print("Starting Naive Bayes")
gnb = GaussianNB()
gnb.fit(x_train, y_train)
end = time.time()
diff=end-start
print("Training time: " + str(diff))
starttest = time.time()
y_pred_nb = gnb.predict(x_test)
endtest =time.time()
difftest = endtest-starttest
print("Test time: " + str(difftest))

#Decision tree
print("Starting Decision tree")
clf = DecisionTreeClassifier()
clf = clf.fit(x_train,y_train)
end = time.time()
diff=end-start
print("Training time: " + str(diff))
starttest = time.time()
y_pred_dt = clf.predict(x_test)
y_pred_dt_roc = clf.predict_proba(x_test)
endtest =time.time()
difftest = endtest-starttest
print("Test time: " + str(difftest))

#Multi layer perceptron
print("Starting Multi layer perceptron")
model = MLPClassifier( max_iter=130, batch_size=1000, alpha=1e-4, activation = 'relu',solver='adam', verbose=10, tol=1e-4, random_state=seed)
model.fit(x_train, y_train)
end = time.time()
diff=end-start
print("Training time: " + str(diff))
starttest = time.time()
y_pred_mlp = model.predict(x_test)
endtest =time.time()
difftest = endtest-starttest
print("Test time: " + str(difftest))

#Gradient boost
print("Starting Gradient boost")
model = GradientBoostingClassifier(n_estimators=20, random_state=seed,verbose=2)
model.fit(x_train, y_train)
end = time.time()
diff=end-start
print("Training time: " + str(diff))
starttest = time.time()
y_pred_gradient = model.predict(x_test)
endtest =time.time()
difftest = endtest-starttest
print("Test time: " + str(difftest))


print("Decision Tree, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_dt)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_dt,average='weighted')))
matrixdt = confusion_matrix(y_test,y_pred_dt)
print(matrixdt)


print("Naive Bayes, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_nb)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_nb,average='weighted')))
matrixnv = confusion_matrix(y_test,y_pred_nb)
print(matrixnv)


print("Neural network, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_nn)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_nn,average='weighted')))
matrixnn = confusion_matrix(y_test,y_pred_nn)
print(matrixnn)

print("MultiLayerPerceptron, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_mlp)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_mlp,average='weighted')))
matrixml = confusion_matrix(y_test,y_pred_mlp)
print(matrixml)
print("Random Forest, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_random)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_random,average='weighted')))
matrixrf = confusion_matrix(y_test,y_pred_random)
print(matrixrf)
print("GradienBoost, accuracy: " + str(metrics.accuracy_score(y_test, y_pred_gradient)) + " F1 score:" + str(metrics.f1_score(y_test, y_pred_gradient,average='weighted')))
matrixgb = confusion_matrix(y_test,y_pred_gradient)
print(matrixgb)


In [ ]:
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt

loss_train = history.history['accuracy']
loss_val = history.history['val_accuracy']
print(loss_train)
print(loss_val)
epochs = range(6)
plt.plot(epochs, loss_train, 'g', label='Training accuracy')
plt.plot(epochs, loss_val, 'b', label='validation accuracy')
plt.title('Training and Validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
dftest.columns

In [ ]:
#DATA CORRELATION
dftest.columns
dftest = dftest.dropna('columns')# drop columns with NaN

dftest = dftest[[col for col in dftest if dftest[col].nunique() > 1]]# keep columns where there are more than 1 unique values

corr = dftest.corr()

plt.figure(figsize=(15,12))

sns.heatmap(corr)

plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})
plt.figure(figsize=[15, 10])
# Data to be plotted
totalDeath = [0.9025883762732175,0.9467405632115038,0.8110844817255842,0.8510844817255842]
totalRecovery = [1000, 1000, 1000, 1000, 1000,1000]
activeCases = [1139958, 347973, 239999, 129360, 34730,34730]
# Using numpy to group 3 different data with bars
X = np.arange(len(totalDeath))
# Passing the parameters to the bar function, this is the main function which creates the bar plot
# Using X now to align the bars side by side
plt.bar(X, totalDeath, color = 'blue', width = 0.25)
# Creating the legend of the bars in the plot
# Overiding the x axis with the country names
plt.xticks([i + 0.0 for i in range(4)], ['NB','DT','NN','RF'],fontsize=18, fontweight='bold')
# Giving the tilte for the plot
plt.title("NB,DT, NN ve RF Karşılaştırma",fontsize=18, fontweight='bold')
# Namimg the x and y axis
plt.ylabel('Doğruluk',fontsize=18, fontweight='bold')
# Saving the plot as a 'png'
plt.savefig('4BarPlot.png')
# Displaying the bar plot
plt.show()

In [ ]:
#Accuracy
fig = plt.figure()
ax = fig.add_axes([0,0.2,0.6,0.8,1])
langs = ['NB','DT','NN','RF']
values = [0.9025883762732175,0.9467405632115038,0.8110844817255842,0.8510844817255842]
f = plt.figure(figsize=(15,3),num=10)
plt.subplot(131)
plt.ylim(0,0.2)
ax.bar(langs,values,width=0.3)
ax.set_ylabel('Doğruluk')
ax.set_title('NB,DT ve NN Karşılaştırma')
plt.show()





In [ ]:
#Accuracy
fig = plt.figure()
ax = fig.add_axes([0,0.2,0.6,0.8,1])
langs = ['NB','DT','NN','RF']
values = [0.9025883762732175,0.9467405632115038,0.8110844817255842,0.8510844817255842]
f = plt.figure(figsize=(15,3),num=10)
plt.subplot(131)
plt.ylim(0,0.2)
ax.bar(langs,values,width=0.3)
ax.set_ylabel('Doğruluk')
ax.set_title('NB,DT ve NN Karşılaştırma')
plt.show()





In [ ]:
#Accuracy
fig = plt.figure()
ax = fig.add_axes([0,0.2,0.6,0.8,1])
langs = ['NB','DT','NN','RF']
values = [0.9025883762732175,0.9467405632115038,0.8110844817255842,0.8510844817255842]
f = plt.figure(figsize=(15,3),num=10)
plt.subplot(131)
plt.ylim(0,0.2)
ax.bar(langs,values,width=0.3)
ax.set_ylabel('Doğruluk')
ax.set_title('NB,DT ve NN Karşılaştırma')
plt.show()





In [ ]:
#F1 Score
fig = plt.figure()
ax = fig.add_axes([0,0,0.8,1])
langs = ['NB','DT','NN']
values = [0.8250899949069051,0.9040449842708569,0.8587280320285047]
f = plt.figure(figsize=(15,3),num=10)
plt.subplot(131)
plt.ylim(0,0.3)
ax.bar(langs,values,width=0.3)
ax.set_ylabel('F1 Skor')
ax.set_title('NB,DT ve NN Karşılaştırma')





In [ ]:
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 22})

plt.figure(figsize=[15, 10])
# Data to be plotted
totalDeath = [0.8250899949069051,0.9040449842708569,0.8587280320285047,0.9040449842708569]
totalRecovery = [1000, 1000, 1000, 1000, 1000,1000]
activeCases = [1139958, 347973, 239999, 129360, 34730,34730]
# Using numpy to group 3 different data with bars
X = np.arange(len(totalDeath))
# Passing the parameters to the bar function, this is the main function which creates the bar plot
# Using X now to align the bars side by side
plt.bar(X, totalDeath, color = 'blue', width = 0.25)
# Creating the legend of the bars in the plot
# Overiding the x axis with the country names
plt.xticks([i + 0.0 for i in range(4)], ['NB','DT','NN','RF'],fontsize=18, fontweight='bold')
# Giving the tilte for the plot
plt.title("NB,DT, NN ve RF Karşılaştırma",fontsize=18, fontweight='bold')
# Namimg the x and y axis
plt.ylabel('F1 Skor',fontsize=18, fontweight='bold')
# Saving the plot as a 'png'
plt.savefig('4BarPlot.png',fontsize=18, fontweight='bold')
# Displaying the bar plot
plt.show()